# Amazon data profiling

https://github.com/j4baek/dse260-CapStone-Amazon

In [1]:
#### To measure all running time
# https://github.com/cpcloud/ipython-autotime

%load_ext autotime

In [2]:
import pandas as pd
import numpy as np 

import json
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

from collections import Counter
from pyspark import SparkContext

time: 506 ms


/home/jh/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Fetch data from S3 bu boto3

https://aws.amazon.com/sdk-for-python/

In [20]:
# !pip install boto3


time: 881 µs


In [23]:
import boto3
import os

s3_client = boto3.client('s3')

def download_dir(prefix, local, bucket, client=s3_client):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)

time: 7.06 ms


In [24]:
!ls

amazon-data-profiling.ipynb	   ratings_Clothing_Shoes_and_Jewelry.html
Clothing_Shoes_and_Jewelry_5.html  README.md
data				   sample_data_1M.html
img				   serverless.yml
LICENSE				   src
time: 195 ms


### Download data from data-lake-landing-zone/raw_data

In [31]:
download_dir('data-lake-landing-zone/raw_data', './data', 'dse-cohort5-group1')

time: 2min 4s


## Descrption of Data set
- https://sites.google.com/a/eng.ucsd.edu/capstone-2020-amazondata/dataset

In [33]:
!ls ./data/data-lake-landing-zone/raw_data

Clothing_Shoes_and_Jewelry_5.json	ratings_Clothing_Shoes_and_Jewelry.pkl
Clothing_Shoes_and_Jewelry_5.pkl	sample_data_1M.pkl
ratings_Clothing_Shoes_and_Jewelry.csv
time: 180 ms


In [37]:
!ls ./data/data-lake-landing-zone/raw_data/

Clothing_Shoes_and_Jewelry_5.json	ratings_Clothing_Shoes_and_Jewelry.pkl
Clothing_Shoes_and_Jewelry_5.pkl	sample_data_1M.pkl
ratings_Clothing_Shoes_and_Jewelry.csv
time: 179 ms


### Set Path and read data

In [ ]:
sample_data_1M_path = './data/data-lake-landing-zone/raw_data/sample_data_1M.json'
Clothing_Shoes_and_Jewelry_5_path='./data/data-lake-landing-zone/raw_data/Clothing_Shoes_and_Jewelry_5.json'
ratings_Clothing_Shoes_and_Jewelry_path='./data/data-lake-landing-zone/raw_data/ratings_Clothing_Shoes_and_Jewelry.csv'

In [3]:
df_sample_data_1M = pd.read_json(sample_data_1M_path, lines=True)

time: 9.32 s


In [4]:
df_Clothing_Shoes_and_Jewelry_5 = pd.read_json(Clothing_Shoes_and_Jewelry_5_path, lines=True)

time: 2.46 s


In [5]:
df_ratings_Clothing_Shoes_and_Jewelry = pd.read_csv(ratings_Clothing_Shoes_and_Jewelry_path)

time: 3.29 s


## Convert json / csv to Pickle

In [15]:
df_sample_data_1M.to_pickle('./data/sample_data_1M.pkl')
test_df = pd.read_pickle('./data/sample_data_1M.pkl')
test_df.shape

(1000000, 12)

time: 4.85 s


In [16]:
df_Clothing_Shoes_and_Jewelry_5.to_pickle('./data/Clothing_Shoes_and_Jewelry_5.pkl')
test_df = pd.read_pickle('./data/Clothing_Shoes_and_Jewelry_5.pkl')
test_df.shape

(278677, 9)

time: 1.25 s


In [17]:
df_ratings_Clothing_Shoes_and_Jewelry.to_pickle('./data/ratings_Clothing_Shoes_and_Jewelry.pkl')
test_df = pd.read_pickle('./data/ratings_Clothing_Shoes_and_Jewelry.pkl')
test_df.shape

(5748919, 4)

time: 2.93 s


## Data profiling

In [6]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df_sample_data_1M, title='sample_data_1M', minimal=True)
profile.to_notebook_iframe()
profile.to_file(output_file="sample_data_1M.html")

time: 4h 7min 40s


In [7]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df_Clothing_Shoes_and_Jewelry_5, title='Clothing_Shoes_and_Jewelry_5', minimal=True)
profile.to_notebook_iframe()
profile.to_file(output_file="Clothing_Shoes_and_Jewelry_5.html")

time: 21.1 s


In [8]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df_ratings_Clothing_Shoes_and_Jewelry, title='ratings_Clothing_Shoes_and_Jewelry', minimal=True)
profile.to_notebook_iframe()
profile.to_file(output_file="ratings_Clothing_Shoes_and_Jewelry.html")

time: 1min 58s
